In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch


In [1]:
# model_directory = '../results/final'

# model = AutoModelForQuestionAnswering.from_pretrained(model_directory)
# tokenizer = AutoTokenizer.from_pretrained(model_directory)
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer, RobertaForQuestionAnswering
from peft import get_peft_model, LoraConfig

# Define the model and checkpoint paths
model_name_or_path = "nur-dev/roberta-kaz-large"  # E.g., "bert-base-uncased"
checkpoint_path = "../results/checkpoint-8000"  # E.g., "path/to/checkpoint"

# Load the model configuration
config = AutoConfig.from_pretrained(model_name_or_path)


tokenizer = AutoTokenizer.from_pretrained("nur-dev/roberta-kaz-large")

# Load the base model
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, config=config)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.2,
    use_dora=True
)
# Load the LoRA weights from checkpoint
class CustomQuestionAnsweringModel(RobertaForQuestionAnswering):
    def __init__(self, masked_model):
        super().__init__(masked_model.config)
        self.roberta.load_state_dict(masked_model.roberta.state_dict())

model = CustomQuestionAnsweringModel(model)
# Wrap the model with LoRA
model = get_peft_model(model, lora_config)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at nur-dev/roberta-kaz-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
context = """
Қазақстан Орталық Азиядағы ең ірі мемлекеттердің бірі болып табылады.
"""
question = "Орталық Азиядағы ең ірі мемлекеттердің бірі?"

# Tokenize the input
inputs = tokenizer.encode_plus(
    question, 
    context, 
    add_special_tokens=True, 
    return_tensors="pt"
)

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Perform inference
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

# Find the answer's start and end position
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

# Decode the answer from the context
answer = tokenizer.decode(input_ids[0][start_index:end_index + 1])

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Орталық Азиядағы ең ірі мемлекеттердің бірі?
Answer: 


In [ ]:
def answer_question(context, question):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    answer = tokenizer.decode(input_ids[0][start_index:end_index + 1])
    return answer

# Создание интерфейса Gradio
interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(lines=10, placeholder="Введите контекст здесь...", label="Контекст"),
        gr.Textbox(lines=1, placeholder="Введите вопрос здесь...", label="Вопрос")
    ],
    outputs="text",
    title="Модель для ответа на вопросы",
    description="Введите контекст и вопрос, чтобы получить ответ на вопрос.",
    examples=[
        ["Қазақстан Орталық Азиядағы ең ірі мемлекеттердің бірі болып табылады.", "Орталық Азиядағы ең ірі мемлекеттердің бірі?"],
        ["Apple — әлемдегі ең қымбат компаниялардың бірі.", "Әлемдегі ең қымбат компания?"]
    ]
)

# Запуск интерфейса
interface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [3]:
from sklearn.metrics import f1_score



def compute_metrics(dataset):
    exact_matches = []
    f1_scores = []

    for data in dataset:
        context = data["context"]
        question = data["question"]
        true_answer_text = data["answer_text"]
        true_answer_start = data["answer_start"]
        true_answer_end = data["answer_end"]

        # Токенизация входных данных
        inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]

        # Вычисление логитов
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

        # Определение начального и конечного индексов предсказанного ответа
        start_index = torch.argmax(start_logits)
        end_index = torch.argmax(end_logits)

        # Декодирование предсказанного ответа
        predicted_answer = tokenizer.decode(input_ids[0][start_index:end_index + 1]).strip()

        # Вычисление метрики Exact Match
        exact_match = 1.0 if predicted_answer == true_answer_text.strip() else 0.0
        exact_matches.append(exact_match)

        # Вычисление F1-оценки
        true_tokens = true_answer_text.split()
        predicted_tokens = predicted_answer.split()
        common_tokens = set(true_tokens) & set(predicted_tokens)
        if len(true_tokens) == 0 or len(predicted_tokens) == 0:
            f1 = 0.0
        else:
            precision = len(common_tokens) / len(predicted_tokens)
            recall = len(common_tokens) / len(true_tokens)
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        f1_scores.append(f1)

    # Средние значения F1 и Exact Match по всему набору данных
    avg_f1 = sum(f1_scores) / len(f1_scores)
    avg_exact_match = sum(exact_matches) / len(exact_matches)

    return {"Average F1": avg_f1, "Average Exact Match": avg_exact_match}


In [4]:
import pandas as pd

df = pd.read_csv('../database/final.csv')

In [6]:
df = df[:1000]
df

,context,question,answer_start,answer_text
0,Belcalis Marlenis Almánzar ( /ˈbɛlkəliːz ɑːlˈm...,Карди Би музыканың қай жанрында ән айтады?,164,рэп
1,Карди Би дискографиясы негізінен трап пен R&am...,Карди Би музыканың қай жанрында ән айтады?,33,трап пен R&amp;B элементтерін қамтитын хип-хоп
2,Тікбұрышты горизонтальды үлендігі бірдей бірін...,Қызыл жолақты американдық жалау нені білдіреді?,537,шыдамдылық пен батырлықты
3,Мо́на Ли́за (Mona Lisa) — бұл шамамен 1503 жыл...,Әйгілі 'Мона Лиза' картинасы қайда қойылған?,217,"Франция, Париж)"
4,Ливерпуль футбол клубы (Liverpool Football Clu...,Ливерпуль Премьер-Лига титулын қанша рет жеңіп...,208,19 рет
...,...,...,...,...
995,==Жел және оның түрлері.==Атмосфералық қысымны...,Жердегі жел ағындарының негізгі энергия көзі қ...,26,Атмосфералық қысымның айырмашылығы
996,«Ақшақар және жеті ергежейлі» (Snow Whiteand t...,Ақшақар фильмі қашан шықты?,374,1937 жылы 21 желтоқсанда
997,"Тетраэдрлер — темір-бетоннан жасалған, биіктіг...",Тетраэдрдің әр бетінің пішіні қандай?,71,пирамида
998,Жазғы Олимпиада ойындары 1896 (1896 Summer Oly...,Алғашқы заманауи Олимпиада ойындары қайда өтті?,270,Афина


In [5]:
df['answer_end'] = df['answer_start'] + df['answer_text'].str.len()
df

,context,question,answer_start,answer_text,answer_end
0,Belcalis Marlenis Almánzar ( /ˈbɛlkəliːz ɑːlˈm...,Карди Би музыканың қай жанрында ән айтады?,164,рэп,167
1,Карди Би дискографиясы негізінен трап пен R&am...,Карди Би музыканың қай жанрында ән айтады?,33,трап пен R&amp;B элементтерін қамтитын хип-хоп,79
2,Тікбұрышты горизонтальды үлендігі бірдей бірін...,Қызыл жолақты американдық жалау нені білдіреді?,537,шыдамдылық пен батырлықты,562
3,Мо́на Ли́за (Mona Lisa) — бұл шамамен 1503 жыл...,Әйгілі 'Мона Лиза' картинасы қайда қойылған?,217,"Франция, Париж)",232
4,Ливерпуль футбол клубы (Liverpool Football Clu...,Ливерпуль Премьер-Лига титулын қанша рет жеңіп...,208,19 рет,214
...,...,...,...,...,...
6635,=== XX ғасыр ===\n\n* 1911 – 12 жылы Түркиямен...,Италияда фашистік партияның жетекшісі кім болған?,753,Б.Муссолини,764
6636,"\nБенито Амилкаре Андреа Муссолини (итал: ""Ben...",Италияда фашистік партияның жетекшісі кім болған?,1,Бенито Амилкаре Андреа Муссолини,33
6637,2-дүниежүзілік соғыс аяқталғаннан кейін Гитлер...,“Қырғи-қабақ соғыс” жылдарындағы негізгі қарсы...,68,КСРО мен АҚШ,80
6638,КСРО мен АҚШ теке тіресі Кариб теңізіндегі дағ...,“Қырғи-қабақ соғыс” жылдарындағы негізгі қарсы...,0,КСРО мен АҚШ,12


In [8]:
metrics = compute_metrics(df.to_dict(orient="records"))
print(metrics)


: 

In [6]:
df = df[:30]
metrics = compute_metrics(df.to_dict(orient="records"))
print(metrics)

{'Average F1': 0.002116402116402116, 'Average Exact Match': 0.0}
